<a href="https://colab.research.google.com/github/quantumhome/DataAnalysisCaseStudy/blob/master/Silver_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Assignment 1*** -
**Use any other open source model in place of openai and do the summerization of pdf document using langchain
and the concept of RAG. Use chromedb vector database for this.**

In [9]:
!pip install -q U langchain langchain-community langchain-classic langchain-google-genai pypdf chromadb langchain-huggingface langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.6 MB/s eta 0

In [10]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from google.colab import userdata

# 1. Environment Setup
os.environ["GOOGLE_API_KEY"] = userdata.get('DeveloperKey')
os.environ["GOOGLE_API_VERSION"] = "v1"

# 2. Document Processing
pdf_file_path = "/content/sample_data/SalesReceipt_58803_126_202599-1022-57.PDF"
loader = PyPDFLoader(pdf_file_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 3. Vector Database
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 4. Initialize LLM (Gemini 2.5 is the stable choice)
# Recommended: "gemini-2.5-flash" (Fastest/Latest)
# Legacy: "gemini-2.5-flash"
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,
    # Ensure you aren't accidentally using the v1beta endpoint
    version="v1"
)

# Test call to verify connectivity
try:
    response = llm.invoke("Are you online?")
    print("Success:", response.content)
except Exception as e:
    print(f"Still failing: {e}")

# 5. Define the Prompt & Build the New Chain
system_prompt = (
    "You are a helpful assistant. Use the provided context to answer the question. "
    "If you don't know the answer based on the context, say you don't know. "
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# Create the internal chain that handles document 'stuffing'
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# Create the full RAG chain
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 6. Execute
query = "Provide a comprehensive summary of the document, focusing on key findings and conclusions."
result = rag_chain.invoke({"input": query})

print(result["answer"])

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: WARNING! version is not default parameter.
                version was transferred to model_kwargs.
                Please confirm that version is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


Success: Yes, I am! I'm an AI model, and I don't really "go offline" in the way a person or a computer might. As long as you can send me messages and I can respond, I am active and available.

How can I help you today?
This document is a receipt issued by **Grow India Nirmiti LLP** to **MR. PIYUSH KUMAR SONI**.

Key findings:

*   **Receipt Number:** 11877
*   **Receipt Date:** September 08, 2025
*   **Customer Details:** MR. PIYUSH KUMAR SONI, residing at B-401, Paras Vista Society, Shedge Vasti, Wakad.
*   **Project Details:** The payment is associated with the project "THE CANARY," specifically for "B BUILDING," Unit B 1203.
*   **Payment Method:** The payment was made via "Online Payment" / "TRANSFER" (Instrument No: 525119731021).
*   **Payment Date:** September 08, 2025
*   **Amount Received:** Rupees Eighty Thousand Only (INR 80,000/-).
*   **Purpose of Payment:** The amount was received as an "Installment With GST."
*   **Disclaimer:** The receipt includes a note stating, "This